In [10]:
import numpy as np
import csv
import pandas as pd
import requests
import json
import time
import yfinance as yf
from datetime import datetime,timedelta

In [11]:
# get df from previously saved df
df=pd.read_csv('../supporting/2009-2022NetIncomeRevenueAndPrice.csv', index_col= 0)
df=df.dropna()
df

,cik,entityName,date,NetIncomeLoss,Revenues,ticker,close
0,910406,"THE HAIN CELESTIAL GROUP, INC.",2009-06-30,-2.472300e+07,1.122734e+09,HAIN,7.805000
1,779152,HENRY JACK & ASSOCIATES INC,2009-06-30,1.031020e+08,7.455930e+08,JKHY,20.750000
2,7084,ARCHER-DANIELS-MIDLAND CO,2009-06-30,1.684000e+09,6.920700e+10,ADM,26.770000
3,16058,CACI International Inc,2009-06-30,8.969800e+07,2.730162e+09,CACI,42.709999
5,820318,II-VI INC,2009-06-30,3.678100e+07,2.922220e+08,IIVI,11.115000
...,...,...,...,...,...,...,...
23970,933034,STRATTEC SECURITY CORP,2022-07-03,7.032000e+06,4.522650e+08,STRT,35.419998
23971,858877,"CISCO SYSTEMS, INC.",2022-07-30,1.181200e+10,5.155700e+10,CSCO,45.049999
23972,1327567,"Palo Alto Networks, Inc",2022-07-31,-2.670000e+08,5.501500e+09,PANW,502.779999
23973,746598,BRADY CORP,2022-07-31,1.499790e+08,1.302062e+09,BRC,48.509998


In [12]:
# we use requests to get infomation from edgar for all tickers and the desired xbrl element
# and then store this in a dictionary
# these are good links about xbrl elements
# https://xbrl.fasb.org/us-gaap/2022/elts/us-gaap-2022.xsd
# http://www.xbrl.org/utr/utr.xml
# https://www.sec.gov/edgar/sec-api-documentation

headers = {
        'User-Agent': 'River Holdings river@gmail.com',
        'Accept-Encoding': 'gzip, deflate',
        'Host': 'data.sec.gov',
    }
response={}
year_range=range(2009,2023)
gaap_shares_elements=['WeightedAverageNumberOfSharesOutstandingBasic']

for year in year_range:
    for index, element in enumerate(gaap_shares_elements):
        URL = f"https://data.sec.gov/api/xbrl/frames/us-gaap/{gaap_shares_elements[index]}/shares/CY{year}.json"
        response[year,element]= requests.get(URL, headers=headers).text
        response[year,element]= response[year,element][450:]
        response[year,element]= response[year,element][response[year,element].find('['):response[year,element].rfind(']')+1]
        print(year, element, len(response[year,element]))
        time.sleep(.1)

2009 WeightedAverageNumberOfSharesOutstandingBasic 459381
2010 WeightedAverageNumberOfSharesOutstandingBasic 701733
2011 WeightedAverageNumberOfSharesOutstandingBasic 809108
2012 WeightedAverageNumberOfSharesOutstandingBasic 781132
2013 WeightedAverageNumberOfSharesOutstandingBasic 753090
2014 WeightedAverageNumberOfSharesOutstandingBasic 724499
2015 WeightedAverageNumberOfSharesOutstandingBasic 680586
2016 WeightedAverageNumberOfSharesOutstandingBasic 646621
2017 WeightedAverageNumberOfSharesOutstandingBasic 626520
2018 WeightedAverageNumberOfSharesOutstandingBasic 609903
2019 WeightedAverageNumberOfSharesOutstandingBasic 661382
2020 WeightedAverageNumberOfSharesOutstandingBasic 707501
2021 WeightedAverageNumberOfSharesOutstandingBasic 680579
2022 WeightedAverageNumberOfSharesOutstandingBasic 15481


In [13]:
# turn the response data into a dictionary of dataframes for each xbrl term and year

df_dict={}

for year in year_range:
    for index, element in enumerate(gaap_shares_elements):
        print(year, element)
        if len(response[year,element]):
            json_object = json.loads(response[year,element])
            df_dict[year,element] = pd.DataFrame.from_dict(json_object)
            df_dict[year,element] = df_dict[year,element].drop(['accn','loc', 'start'], axis='columns')
            df_dict[year,element]=df_dict[year,element].rename(columns={'end':'date', 'val':f'{element}'})
print(df_dict[2009,'WeightedAverageNumberOfSharesOutstandingBasic'].head())
print(len(df_dict[2009,'WeightedAverageNumberOfSharesOutstandingBasic']))

2009 WeightedAverageNumberOfSharesOutstandingBasic
2010 WeightedAverageNumberOfSharesOutstandingBasic
2011 WeightedAverageNumberOfSharesOutstandingBasic
2012 WeightedAverageNumberOfSharesOutstandingBasic
2013 WeightedAverageNumberOfSharesOutstandingBasic
2014 WeightedAverageNumberOfSharesOutstandingBasic
2015 WeightedAverageNumberOfSharesOutstandingBasic
2016 WeightedAverageNumberOfSharesOutstandingBasic
2017 WeightedAverageNumberOfSharesOutstandingBasic
2018 WeightedAverageNumberOfSharesOutstandingBasic
2019 WeightedAverageNumberOfSharesOutstandingBasic
2020 WeightedAverageNumberOfSharesOutstandingBasic
2021 WeightedAverageNumberOfSharesOutstandingBasic
2022 WeightedAverageNumberOfSharesOutstandingBasic
       cik                 entityName        date  \
0   864328             BJ SERVICES CO  2009-09-30   
1  1076405   PEPSI BOTTLING GROUP INC  2009-12-26   
2  1335793               CNX GAS CORP  2009-12-31   
3   804055  COCA COLA ENTERPRISES INC  2009-12-31   
4   721083    SMITH I

In [14]:
# build a single dataframe with all the data by date and cik number
# use different df than the one previously loaded, we will join in the next cell

df2=pd.DataFrame()

for index, element in enumerate(gaap_shares_elements):
    df_dict[element]= pd.DataFrame()
    for year in year_range:
        try:
            df_dict[element]= pd.concat([df_dict[element],df_dict[year,element]])
        except Exception as e:
            print(e)

for index, element in enumerate(gaap_shares_elements):
    if index==0:
        df2=df_dict[element]
    else:
        try:
            df2=df2.join(df_dict[element].set_index(['cik','entityName','date']), on=['cik','entityName','date'])
        except:
            print('Your year range is likely missing an element')
df2=df2.rename(columns={'WeightedAverageNumberOfSharesOutstandingBasic':'shares'})
df2

,cik,entityName,date,shares
0,864328,BJ SERVICES CO,2009-09-30,292239000.0
1,1076405,PEPSI BOTTLING GROUP INC,2009-12-26,216000000.0
2,1335793,CNX GAS CORP,2009-12-31,150977235.0
3,804055,COCA COLA ENTERPRISES INC,2009-12-31,488000000.0
4,721083,SMITH INTERNATIONAL INC,2009-12-31,222353000.0
...,...,...,...,...
97,1808898,Benitec Biopharma Inc.,2022-06-30,8171690.0
98,1815974,"ANEBULO PHARMACEUTICALS, INC.",2022-06-30,23344567.0
99,1820953,"Affirm Holdings, Inc.",2022-06-30,281704041.0
100,1828376,WILLIAM PENN BANCORPORATION,2022-06-30,14255901.0


In [15]:
# join dfs
df3=df.join(df2.set_index(['cik','entityName','date']), on=['cik','entityName','date'])
df3

,cik,entityName,date,NetIncomeLoss,Revenues,ticker,close,shares
0,910406,"THE HAIN CELESTIAL GROUP, INC.",2009-06-30,-2.472300e+07,1.122734e+09,HAIN,7.805000,4.048300e+07
1,779152,HENRY JACK & ASSOCIATES INC,2009-06-30,1.031020e+08,7.455930e+08,JKHY,20.750000,8.411800e+07
2,7084,ARCHER-DANIELS-MIDLAND CO,2009-06-30,1.684000e+09,6.920700e+10,ADM,26.770000,6.430000e+08
3,16058,CACI International Inc,2009-06-30,8.969800e+07,2.730162e+09,CACI,42.709999,2.997600e+07
5,820318,II-VI INC,2009-06-30,3.678100e+07,2.922220e+08,IIVI,11.115000,5.933400e+04
...,...,...,...,...,...,...,...,...
23970,933034,STRATTEC SECURITY CORP,2022-07-03,7.032000e+06,4.522650e+08,STRT,35.419998,3.861000e+06
23971,858877,"CISCO SYSTEMS, INC.",2022-07-30,1.181200e+10,5.155700e+10,CSCO,45.049999,4.170000e+09
23972,1327567,"Palo Alto Networks, Inc",2022-07-31,-2.670000e+08,5.501500e+09,PANW,502.779999,9.850000e+07
23973,746598,BRADY CORP,2022-07-31,1.499790e+08,1.302062e+09,BRC,48.509998,5.132100e+07


In [16]:
# do a little renaming and cleanup
df3=df3.rename(str.lower, axis='columns')
df3=df3.rename(columns={'netincomeloss':'netincome','entityname':'name','revenues':'revenue'})
df3['pe']=df3['shares']*df3['close']/df3['revenue']
df3=df3[['cik', 'ticker', 'name', 'date', 'shares','netincome','revenue','pe','close']]
df3

,cik,ticker,name,date,shares,netincome,revenue,pe,close
0,910406,HAIN,"THE HAIN CELESTIAL GROUP, INC.",2009-06-30,4.048300e+07,-2.472300e+07,1.122734e+09,0.281429,7.805000
1,779152,JKHY,HENRY JACK & ASSOCIATES INC,2009-06-30,8.411800e+07,1.031020e+08,7.455930e+08,2.341021,20.750000
2,7084,ADM,ARCHER-DANIELS-MIDLAND CO,2009-06-30,6.430000e+08,1.684000e+09,6.920700e+10,0.248719,26.770000
3,16058,CACI,CACI International Inc,2009-06-30,2.997600e+07,8.969800e+07,2.730162e+09,0.468937,42.709999
5,820318,IIVI,II-VI INC,2009-06-30,5.933400e+04,3.678100e+07,2.922220e+08,0.002257,11.115000
...,...,...,...,...,...,...,...,...,...
23970,933034,STRT,STRATTEC SECURITY CORP,2022-07-03,3.861000e+06,7.032000e+06,4.522650e+08,0.302382,35.419998
23971,858877,CSCO,"CISCO SYSTEMS, INC.",2022-07-30,4.170000e+09,1.181200e+10,5.155700e+10,3.643705,45.049999
23972,1327567,PANW,"Palo Alto Networks, Inc",2022-07-31,9.850000e+07,-2.670000e+08,5.501500e+09,9.001878,502.779999
23973,746598,BRC,BRADY CORP,2022-07-31,5.132100e+07,1.499790e+08,1.302062e+09,1.912030,48.509998


In [ ]:
# save as csv
df3.to_csv("edgar-stock-analysis/edgar-data/supporting/2009-2022RevenueIncomePePrice.csv")
